# **Neural steering controller for autonomous parking**

This tutorial implements the physics-driven neural steering controller of the paper titled:

*Fast Planning and Tracking of Complex Autonomous Parking Maneuvers With Optimal Control and Pseudo-Neural Networks*

(available at https://ieeexplore.ieee.org/abstract/document/10309845)

## Initialization

### Import packages

In [185]:
import sys
import os
import torch
import pandas as pd
import scipy as sp

print('Current working directory: ',os.getcwd())
sys.path.append(os.path.join(os.getcwd(),'..'))
from neu4mes import *
from neu4mes import relation
from neu4mes import earlystopping
relation.NeuObj_names = []  # reset the list of NeuObj names

# import a library for plots
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
#mpl.rcParams['text.usetex'] = True
import matplotlib.pyplot as plt
plt.close('all')
SMALL_SIZE = 14
MEDIUM_SIZE = 22
BIGGER_SIZE = 26
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('grid', linestyle="--", color='grey')

# enable zooming on the plots
%matplotlib inline
import mpld3
mpld3.enable_notebook()


Current working directory:  /Users/mattiapiccinini/Documents/Research/Neu4Mes/tutorials


### Configurations, known constant parameters and initial guesses

In [186]:
# Path to the data folder
data_folder = os.path.join(os.getcwd(),'datasets','control_steer_car_parking')

# Import the file with the vehicle data
vehicle_data_csv = os.path.join(data_folder,'other_data','vehicle_data.csv')
# Extract the vehicle data
L = pd.read_csv(vehicle_data_csv)['L'][0]  # [m] wheelbase

# Import the file with the steering maps
steer_maps_file = os.path.join(data_folder,'other_data','steer_map.txt')
steer_map_load  = np.loadtxt(steer_maps_file, delimiter='\t', skiprows=1)
delta_w_avg_map = np.float64(np.deg2rad(steer_map_load[:,0]))  # [rad] average steering angle at the front wheels
delta_sw_map    = np.float64(np.deg2rad(steer_map_load[:,1]))  # [rad] steering wheel angle

# Initial guesses
# Load the initial guesses for the curvature diagram approximation, computed in Matlab with a 2nd order optimizer
initial_guesses_curv_diagr = sp.io.loadmat(os.path.join(data_folder,'other_data','initial_guesses_curv_diagram','fit_curv_diagr_5th_ord_poly.mat'))
curv_diagram_params_guess = initial_guesses_curv_diagr['optim_params_poly'][0].astype('float64')

## NN model

### Custom parametric functions

In [187]:
# Curvature diagram
def curvat_diagram(curv,L,h_1,h_2,h_3):
  return torch.atan(h_1*curv + h_2*torch.pow(curv,3) + h_3*torch.pow(curv,5) + curv*L)

# Steering maps
def steer_map_spline(x,x_data,y_data):
  # Inputs: 
  # x: average steering angle at the front wheels [rad]
  # x_data: map of average steering angles at the front wheels (delta_w_avg_map) [rad]
  # y_data: map of steering wheel angles (delta_sw_map) [rad]
  # Output:
  # y: steering wheel angle [rad]

  x_data = x_data[0].squeeze()
  y_data = y_data[0].squeeze()

  # Linear interpolation of the steering map:
  # Loop over the batch dimension
  for i in range(x.shape[0]):
    # Find the indices of the intervals containing each x
    indices = torch.searchsorted(x_data, x[i], right=True).clamp(1, len(x_data) - 1)
    
    # Get the values for the intervals
    x1 = x_data[indices - 1]
    x2 = x_data[indices]
    y1 = y_data[indices - 1]
    y2 = y_data[indices]
    
    # Linear interpolation formula
    y = y1 + (y2 - y1) * (x[i] - x1) / (x2 - x1)
    
    # Saturate the output if x is out of bounds
    y = torch.where(x[i] < x_data[0], y_data[0], y)    # Saturate to minimum y_data
    y = torch.where(x[i] > x_data[-1], y_data[-1], y)  # Saturate to maximum y_data
    if i == 0:
      y_batch = y
    else:
      y_batch = torch.cat((y_batch,y),0)
  y_batch = y_batch.unsqueeze(1)
  return y_batch

### Internal architecture

In [188]:
# Neural model inputs and outputs
curv         = Input('curv')          # [1/m] path curvature
steer        = Input('steer')         # [rad] steering wheel angle 
steer_target = Input('steer_target')  # [rad] steering wheel angle --> this is used only as a training target

num_samples_future_curv = 15   # number of samples in the future for the curvature prediction
num_samples_past_steer  = 15   # number of samples in the past for the steering wheel angle prediction

# Trainable parameters:
# Curvature diagram parameters
h_1_guess = Parameter('h_1',values=[[curv_diagram_params_guess[0]]])  # initial guess
h_2_guess = Parameter('h_2',values=[[curv_diagram_params_guess[1]]])  # initial guess
h_3_guess = Parameter('h_3',values=[[curv_diagram_params_guess[2]]])  # initial guess

# Parametric function to learn the curvature diagram 
out_curv_diagr = ParamFun(curvat_diagram,parameters=[h_1_guess,h_2_guess,h_3_guess])(curv.sw([0,num_samples_future_curv]),L.item())

# FIR layer to weigh the future predictions of the curvature diagram
out_fir        = Fir(parameter='fir_future_curv', parameter_init=init_negexp, 
                     parameter_init_params={'size_index':0, 'first_value':0.1, 'lambda':5})(out_curv_diagr)   

# Parametric function to model the steering map (i.e., the relation between the average steering angle at the front wheels and the steering wheel angle)
out_steer_map  = ParamFun(steer_map_spline)(out_fir,[list(delta_w_avg_map)],[list(delta_sw_map)])     

# FIR layer to weigh the past steering wheel angles computed by the NN (auto-regressive model)
out_arx        = Fir(parameter='fir_auto_regression', parameter_init=init_negexp, 
                     parameter_init_params={'size_index':0, 'first_value':1e-3, 'lambda':5})(steer.sw([-num_samples_past_steer,0]))  

# Output of the neural model
out = Output('steering_angle', out_steer_map + out_arx)

### Neu4Mes framework

In [189]:
# Create a neu4mes model
steer_controller_park = Neu4mes(visualizer='Standard',seed=0,workspace=os.path.join(os.getcwd(),'trained_models'))  #visualizer=MPLVisulizer()

# Add the neural model to the neu4mes structure and neuralization of the model
steer_controller_park.addModel('steer_ctrl',[out])
steer_controller_park.addMinimize('steer_error', 
                                  steer_target.next(),  # next means the first value in the "future"
                                  out, 
                                  loss_function='mse')
steer_controller_park.neuralizeModel()

================================ Neu4mes Model =================================
{'Constants': {'Constant286': {'dim': 1, 'values': 2.6},
               'Constant289': {'dim': 100,
                               'sw': 1,
                               'values': [[-0.6417302839400707,
                                           -0.6287660357796734,
                                           -0.6158017876192587,
                                           -0.6028375394588616,
                                           -0.589873291298447,
                                           -0.5769090431380498,
                                           -0.5639447949776352,
                                           -0.5509805468172381,
                                           -0.5380162986568233,
                                           -0.5250520504964262,
                                           -0.512087802336029,
                                           -0.4991235541756144,
             

## Training and validation datasets

In [190]:
# Load the training and the validation dataset
data_struct = ['curv',('steer','steer_target')]  # both steer and steer_target are read from the same column of the csv file
data_folder_train = os.path.join(data_folder,'training')
data_folder_valid = os.path.join(data_folder,'validation')
data_folder_test  = os.path.join(data_folder,'test')
steer_controller_park.loadData(name='training_set', source=data_folder_train, format=data_struct, skiplines=1)
steer_controller_park.loadData(name='validation_set', source=data_folder_valid, format=data_struct, skiplines=1)
steer_controller_park.loadData(name='test_set', source=data_folder_test, format=data_struct, skiplines=1)

# check the definition of the windows in the inputs and outputs
#samples_test_set = steer_controller_park.get_samples('training_set', index=100, window=1) 
#print(samples_test_set)

============================ Neu4mes Model Dataset =============================
Dataset Name:                 training_set
Number of files:              1
Total number of samples:      3092
Shape of steer:               (3092, 15, 1)
Shape of curv:                (3092, 15, 1)
Shape of steer_target:        (3092, 1, 1)
============================ Neu4mes Model Dataset =============================
Dataset Name:                 validation_set
Number of files:              1
Total number of samples:      832
Shape of steer:               (832, 15, 1)
Shape of curv:                (832, 15, 1)
Shape of steer_target:        (832, 1, 1)
============================ Neu4mes Model Dataset =============================
Dataset Name:                 test_set
Number of files:              1
Total number of samples:      691
Shape of steer:               (691, 15, 1)
Shape of curv:                (691, 15, 1)
Shape of steer_target:        (691, 1, 1)


## Training

### Train the NN in open-loop (no auto-regressive term)

In [191]:
num_epochs = 20
batch_size = 100
learn_rate = 1e-3  # learning rate
early_stop_patience = 100
training_pars_open_loop = {'num_of_epochs':num_epochs, 
                           'val_batch_size':batch_size, 
                           'train_batch_size':batch_size, 
                           'lr':learn_rate}

steer_controller_park.trainModel(train_dataset='training_set', validation_dataset='validation_set', 
                                 training_params=training_pars_open_loop, optimizer='Adam', shuffle_data=True,
                                 early_stopping=earlystopping.early_stop_valid_patience, early_stopping_params={'patience':early_stop_patience})  

======================== Neu4mes Model Train Parameters ========================
models:                       ['steer_ctrl']
train dataset:                training_set
train {batch size, samples}:  {100, 3092}
val dataset:                  validation_set
val {batch size, samples}:    {100, 832}
num of epochs:                20
shuffle data:                 True
early stopping:               early_stop_valid_patience
early stopping params:        {'patience': 100}
minimize:                     {'steer_error': {'A': 'SamplePart479',
                                               'B': 'steering_angle',
                                               'loss': 'mse'}}
optimizer:                    Adam
optimizer defaults:           {'lr': 0.001}
optimizer params:             [{'params': 'fir_auto_regression'},
                               {'params': 'fir_future_curv'},
                               {'params': 'h_1'},
                               {'params': 'h_2'},
                      

({'steer_error': [7.305891036987305,
   0.48317310214042664,
   0.27032148838043213,
   0.1838829219341278,
   0.1233048364520073,
   0.0836595743894577,
   0.06025420501828194,
   0.04668377712368965,
   0.04021519795060158,
   0.03659301996231079,
   0.03540443629026413,
   0.033661697059869766,
   0.03330451622605324,
   0.033495981246232986,
   0.033236369490623474,
   0.03298412263393402,
   0.03248501941561699,
   0.032533980906009674,
   0.032196544110774994,
   0.0319557711482048]},
 {'steer_error': [0.6748749613761902,
   0.5277137756347656,
   0.40599745512008667,
   0.2879498600959778,
   0.20993712544441223,
   0.16258849203586578,
   0.1316787600517273,
   0.11979814618825912,
   0.11011503636837006,
   0.10767537355422974,
   0.10406070202589035,
   0.10315696895122528,
   0.10303408652544022,
   0.10204465687274933,
   0.10156626254320145,
   0.10075300186872482,
   0.09973315894603729,
   0.10103550553321838,
   0.09919629991054535,
   0.09787242114543915]},
 {})

In [192]:
# Print the trained NN parameters
steer_controller_park.neuralizeModel()

================================ Neu4mes Model =================================
{'Constants': {'Constant286': {'dim': 1, 'values': 2.6},
               'Constant289': {'dim': 100,
                               'sw': 1,
                               'values': [[-0.6417302839400707,
                                           -0.6287660357796734,
                                           -0.6158017876192587,
                                           -0.6028375394588616,
                                           -0.589873291298447,
                                           -0.5769090431380498,
                                           -0.5639447949776352,
                                           -0.5509805468172381,
                                           -0.5380162986568233,
                                           -0.5250520504964262,
                                           -0.512087802336029,
                                           -0.4991235541756144,
             

### Re-train the NN in auto-regressive mode (closed-loop training)

In [193]:
num_epochs = 2
batch_size = 100
learn_rate = 1e-3  # learning rate
early_stop_patience = 60
training_pars_closed_loop = {'num_of_epochs':num_epochs, 
                             'val_batch_size':batch_size, 
                             'train_batch_size':batch_size, 
                             'lr':learn_rate}

predict_samples = 50  # number of samples after which the internal state is reset
steps_skip = 1  # number of samples to skip when going to a new window. The default is 1, meaning the size of a batch. If steps_skip = predict_samples, then the whole window size is skipped

# NOTE: by default, the next batch skips a full length of a batch
# NOTE: shuffle = True shuffles only the order of the batches, so it's ok with the autoregression

steer_controller_park.trainModel(train_dataset='training_set', validation_dataset='validation_set', 
                                 training_params=training_pars_closed_loop, optimizer='Adam', shuffle_data=True,
                                 early_stopping=earlystopping.early_stop_valid_patience, early_stopping_params={'patience':early_stop_patience},
                                 prediction_samples=predict_samples, step=steps_skip, closed_loop={'steer':'steering_angle'})  

Recurrent train: closing the loop between the the input ports steer and the output ports steering_angle for 50 samples
======================== Neu4mes Model Train Parameters ========================
models:                       ['steer_ctrl']
train dataset:                training_set
train {batch size, samples}:  {100, 3092}
val dataset:                  validation_set
val {batch size, samples}:    {100, 832}
num of epochs:                2
shuffle data:                 True
early stopping:               early_stop_valid_patience
early stopping params:        {'patience': 60}
minimize:                     {'steer_error': {'A': 'SamplePart479',
                                               'B': 'steering_angle',
                                               'loss': 'mse'}}
prediction samples:           50
step:                         1
closed loop:                  {'steer': 'steering_angle'}
connect:                      {}
optimizer:                    Adam
optimizer defaults:  

({'steer_error': [0.11733747273683548, 0.07187088578939438]},
 {'steer_error': [0.21352608501911163, 0.17652063071727753]},
 {})

### Print the parameters of the trained NN

In [194]:
# Print the trained NN parameters
steer_controller_park.neuralizeModel()

================================ Neu4mes Model =================================
{'Constants': {'Constant286': {'dim': 1, 'values': 2.6},
               'Constant289': {'dim': 100,
                               'sw': 1,
                               'values': [[-0.6417302839400707,
                                           -0.6287660357796734,
                                           -0.6158017876192587,
                                           -0.6028375394588616,
                                           -0.589873291298447,
                                           -0.5769090431380498,
                                           -0.5639447949776352,
                                           -0.5509805468172381,
                                           -0.5380162986568233,
                                           -0.5250520504964262,
                                           -0.512087802336029,
                                           -0.4991235541756144,
             

## Test on a new dataset

In [195]:
# Test on a new dataset
samples_test_set = steer_controller_park.get_samples('validation_set', index=0, window=50) 
steer_controller_park.resetStates()  # reset the internal state
out_nn_test_set  = steer_controller_park(samples_test_set)

# Test with custom data
#steer_controller_park({'curv':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],'steer':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]})

Different number of samples between inputs [MAX steer_target = 50; MIN curv = 36]


## Export the trained NN

In [196]:
# Export the model
steer_controller_park.neuralizeModel()
steer_controller_park.exportJSON()

flag_load_trained_model = False
if flag_load_trained_model:
  # Reload the trained model:
  # Load the json file with the model
  json_folder = os.path.join(os.getcwd(),'tutorials','trained_models','neu4mes_2024_10_07_17_50')
  json_file = os.path.join(json_folder,'model.json')
  import json
  # Open and read the JSON file
  with open(json_file, 'r') as file:
      model_trained_json = json.load(file)

  steer_controller_park.model_def = model_trained_json
  # steer_controller_park.trainModel(train_dataset='training_set', validation_dataset='validation_set', 
  #                                  training_params=training_pars_closed_loop, optimizer='Adam', shuffle_data=True,
  #                                  prediction_samples=predict_samples, step=steps_skip,
  #                                  early_stopping=earlystopping.early_stop_valid_patience, early_stopping_params={'exit_tol':1-3})  

================================ Neu4mes Model =================================
{'Constants': {'Constant286': {'dim': 1, 'values': 2.6},
               'Constant289': {'dim': 100,
                               'sw': 1,
                               'values': [[-0.6417302839400707,
                                           -0.6287660357796734,
                                           -0.6158017876192587,
                                           -0.6028375394588616,
                                           -0.589873291298447,
                                           -0.5769090431380498,
                                           -0.5639447949776352,
                                           -0.5509805468172381,
                                           -0.5380162986568233,
                                           -0.5250520504964262,
                                           -0.512087802336029,
                                           -0.4991235541756144,
             